# This implementation of the genetic algorithm for the science objectives has the exact seed used for MATER's requriements

This is meant to demonstrate replication and is not the way the this seed was found

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [4]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime

# Loads in the analog payloads and the science objective scores

In [5]:
#import cell

#science
science_array = []

#take data from experiments csv and convert it to an array
with open('Found_Experiments.csv', 'rt') as f:
    array_y = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_y:
        science_array.append(line)
#print(science_array)


#take data from scores science csv and convert it to an array
objectives_score = []
with open('Objective_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        objectives_score.append(line)


# Prepares data from the .csv files and sets parameter values

In [7]:
#data processing cell

#Parse objective data
science_array_array_mod = science_array[1:]                                         #cut off title score
science_mass_array      = np.asarray([i[-1] for i in science_array_array_mod],float)      #Masses of experiments 0 indexed
science_power_array     = np.asarray([i[-2] for i in science_array_array_mod],float)      #power consumption of experiments 0 indexed
objective_array_temp    = [np.array(i[1:-2]) for i in science_array_array_mod]  
objective_array         = []
for i in objective_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    objective_array.append(arr)
objective           = np.array(objectives_score[1:])
objective           = np.asarray(objective[0:, 1:],float)
objective_matrix    = np.zeros((len(objective_array),len(objective)))
for i in range(len(objective_matrix)):
    for j in objective_array[i]:
        objective_matrix[i][j-1] = 1

objective_matrix = np.transpose(objective_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

objective_Understanding = objective[0:, 1]
objective_Humans        = objective[0:, 2]

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function being optimized

In [8]:
#score function for science

def score_science(mission, check_mass=True):
    #weights
    w1 = 4  #completed_score
    w2 = 5  #understand_score
    w3 = 7  #human_score

    mass_max_science = max_mass*mass_ratio #in kg
    mass_min_science = min_mass*mass_ratio #in kg

    #Mass check
    if check_mass and (science_mass_array.dot(mission) + beta*science_power_array.dot(mission) <= mass_min_science or science_mass_array.dot(mission) + beta*science_power_array.dot(mission) > mass_max_science):
        return abs(science_mass_array.dot(mission) + beta*science_power_array.dot(mission))

    #normlizers
    max_number      = 46       #normalize number score 0-1
    max_understand  = 5        #normlize understanding score 0-1
    max_human       = 5        #normlize human score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(objective_matrix @ mission, a_min=None, a_max=1) #A objective is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of objective completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_understanding_score   = objective_Understanding @ normalized_completed       #Generates the understanding score
    completed_human_score           = objective_Humans   @ normalized_completed            #Generates the human score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_understanding_score/max_understand + w3*completed_human_score/max_human
    return -score


# Initial Guess

In [9]:
#analytical solution
scores_per_sci = np.zeros(len(science_array)-1)
for i in range(len(scores_per_sci)):
    input = np.zeros(len(science_array)-1)
    input[i] = 1
    scores_per_sci[i] = score_science(input,check_mass=False)
indcies_tech = np.argsort(scores_per_sci)
scores_per_tech_sort = np.sort(scores_per_sci) 

#rint(scores_per_tech_sort)
#print(indcies_tech)
mass = 0
bring_array_science = np.zeros(len(science_array)-1)
for i in range(len(indcies_tech)):
    if mass < 0.5*(max_mass+min_mass)*(mass_ratio):
        bring_array_science[indcies_tech[i]] = 1
        mass = science_mass_array.dot(bring_array_science) + beta*science_power_array.dot(bring_array_science)

print(bring_array_science)
print(score_science(bring_array_science))
#print(mass)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
-7.619983277591974


# Differential Evolution call

In [10]:
#Optimizer for science https://en.wikipedia.org/wiki/Differential_evolution https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution
bounds_science  = Bounds(lb=np.zeros(len(objective_array)), ub=np.ones(len(objective_array)))                                                       #we either bring or we dont
seed            = 6445774 #np.random.randint(9999999)
res_science     = differential_evolution(func=score_science, bounds=bounds_science, x0=bring_array_science, integrality=np.ones(len(science_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=1000,seed=seed)
print(res_science)
print('seed: ' + str(seed))

             message: Optimization terminated successfully.
             success: True
                 fun: -7.741847826086957
                   x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                 nit: 656
                nfev: 1345536
          population: [[ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                       [ 0.000e+00  1.000e+00 ...  1.000e+00  0.000e+00]
                       ...
                       [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                       [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]]
 population_energies: [-7.742e+00 -7.618e+00 ... -7.404e+00 -7.569e+00]
seed: 6445774


# Presents and saves outputs

In [12]:
#Presenter for science

x_science = res_science['x']
x_science = x_science.astype(int)
experiments_brought = []
j = 0
mass_science = 0
power_science = 0
for i in range(len(x_science)):
    if x_science[i] == 1:
        experiments_brought.append(science_array_array_mod[i])
        mass_science    += science_mass_array[i]
        power_science   += science_power_array[i]
        j += 1
mass_science    = round(mass_science, 3)
power_science   = round(power_science, 3)
science_score   = round(-score_science(x_science), 3)
print("For science experiments:")
print()
print(" Number of Payloads              :   " + str(len(experiments_brought)))
print(" Mass of Mission                 :   " + str(mass_science))
print(" Power Consumption of Mission    :   " + str(power_science))
print(" Combined Mass of Mission        :   " + str(mass_science + beta*power_science))
print(" Score of Mission                :   " + str(science_score))
print()
print(" Payloads:")
for i in experiments_brought:
    print("     " +i[0])

now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
#write to csv
with open("raw_science_out_"  + formatted_time +  '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    wr.writerow(["", "Name", "Objectives", "Power", "Mass", "Total Mass"])
    for i in experiments_brought:
        arr = []
        arr.append("")
        arr.append(i[0])
        arr.append(i[1:-2])
        arr.append(i[-2])
        arr.append(i[-1])
        wr.writerow(arr)
    wr.writerow(["Totals", len(experiments_brought), science_score, power_science, mass_science, mass_science + beta*power_science, 'seed: ', seed])

For science experiments:

 Number of Payloads              :   19
 Mass of Mission                 :   9652.087
 Power Consumption of Mission    :   11.919
 Combined Mass of Mission        :   11857.101999999999
 Score of Mission                :   7.742

 Payloads:
     Rock Abrasion Tool (RAT)
     inchworm driling system (IEEE) 
     rover 2
     rover 3
     Rover4
     rover with sensors
     radiation sensoring
     Rover 7
     Rover 8
     Rover 10
     rover to make oxygen
     rover with sensors 2
     radiation sensoring 2
     rover 21
     Insight
     Atmo rover
     spectroscopy using Super Cam on Perseverance
     Drilling Rover
     Mass Spectrometer
